download the pop909 dataset as a zip file and upload it directly to this environment

preprocess and augment the dataset


In [ ]:
!unzip -q POP909-Dataset-master.zip -d POP909
!mv /content/POP909/POP909-Dataset-master/POP909.zip* /content/
!rm -rf /content/POP909
!rm -rf /content/POP909-Dataset-master.zip
!unzip -q POP909.zip -d POP909
!mv /content/POP909/POP909/* /content/POP909/
!rm -rf /content/POP909.zip
!rm -rf /content/POP909/index.xlsx

Import statements, global variables

In [ ]:
!pip install mido

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.1
    Uninstalling packaging-24.1:
      Successfully uninstalled packaging-24.1


In [ ]:
import mido
import os
import numpy as np
import shutil

In [ ]:
# base_dir should just be declared once here
base_dir = '/content/POP909'

isolate melody track, save as midi file

In [ ]:
def view_midi_tracks(midi_file_path):
  mid = mido.MidiFile(midi_file_path)
  print(f"Number of tracks: {len(mid.tracks)}")
  for i, track in enumerate(mid.tracks):
    print(f"Track {i}: {track.name}")

def isolate_track(midi_file_path, track_index, output_file_path):
  mid = mido.MidiFile(midi_file_path)
  if track_index < 0 or track_index >= len(mid.tracks):
    print("Invalid track index")
    return
  new_mid = mido.MidiFile()
  new_mid.tracks.append(mid.tracks[track_index])
  new_mid.save(output_file_path)
  print(f"Track {track_index} saved to {output_file_path}")

def process_pop909_dataset(base_dir):
  for song_id in range(1, 910):  # 001 to 909 lol
    song_dir = os.path.join(base_dir, f"{song_id:03d}")
    input_file = os.path.join(song_dir, f"{song_id:03d}.mid")
    output_file = os.path.join(song_dir, f"{song_id:03d}_melody_isolated.mid")

    if os.path.exists(input_file):
      print(f"Processing song {song_id:03d}")
      view_midi_tracks(input_file)
      isolate_track(input_file, 1, output_file)
    else:
      print(f"File not found: {input_file}")

In [ ]:
process_pop909_dataset(base_dir)

convert isolated melodies to textual representation, in 16th notes

In [ ]:
def read_beat_audio(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    times = [float(line.split()[0]) for line in lines]
    beats = [float(line.split()[1]) for line in lines]
    return times, beats

def calculate_tempo(times, beats):
    measure_durations = [times[i+4] - times[i] for i in range(0, len(times)-4, 4)]
    avg_measure_duration = np.mean(measure_durations)
    return 60 / (avg_measure_duration / 4)  # BPM

def midi_to_text_sequence(midi_file, beat_audio_file):
    # Read MIDI file
    mid = mido.MidiFile(midi_file)

    # Read beat audio file and calculate tempo
    times, beats = read_beat_audio(beat_audio_file)
    bpm = calculate_tempo(times, beats)

    # Calculate duration of a 16th note in seconds
    seconds_per_beat = 60 / bpm
    seconds_per_16th = seconds_per_beat / 4

    # Initialize variables
    current_time = 0
    current_16th = 0
    max_16th = int(mid.length / seconds_per_16th) + 1
    sequence = ["_"] * max_16th
    active_notes = {}

    for msg in mid:
        current_time += msg.time
        new_16th = int(current_time / seconds_per_16th)

        # Handle note continuations
        for note, start in list(active_notes.items()):
            for i in range(current_16th + 1, new_16th):
                if i > start:
                    sequence[i] = "-"

        current_16th = new_16th

        if msg.type == 'note_on' and msg.velocity > 0:
            sequence[current_16th] = str(msg.note)
            active_notes[msg.note] = current_16th
        elif msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0):
            if msg.note in active_notes:
                del active_notes[msg.note]

    # Process the sequence to remove excessive silence
    def remove_excess_silence(seq):
        # Remove leading silence
        while seq and seq[0] == "_":
            seq.pop(0)

        # Remove sequences of more than 8 "_" symbols
        i = 0
        while i < len(seq):
            if seq[i] == "_":
                j = i + 1
                while j < len(seq) and seq[j] == "_":
                    j += 1
                if j - i > 8:
                    del seq[i:j]
                    i = i  # Stay at the current position
                else:
                    i = j  # Move to the next non-"_" character
            else:
                i += 1

        return seq

    # Remove excess silence from the sequence
    sequence = remove_excess_silence(sequence)

    return sequence

# Usage

def process_dataset_ttm(base_dir):
    for song_id in range(1, 910):  # 001 to 909
        song_dir = os.path.join(base_dir, f"{song_id:03d}")
        input_file = os.path.join(song_dir, f"{song_id:03d}_melody_isolated.mid")
        beat_audio_file = os.path.join(song_dir, "beat_audio.txt")
        output_file = os.path.join(song_dir, f"{song_id:03d}_melody.txt")

        if os.path.exists(input_file):
            print(f"Processing song {song_id:03d}")
            result = midi_to_text_sequence(input_file, beat_audio_file)
            # Convert the result list to a string
            result_string = " ".join(result)
            # Write the result to the output file
            with open(output_file, 'w') as f:
                f.write(result_string)
            print(f"Output written to: {output_file}")
        else:
            print(f"File not found: {input_file}")

In [ ]:
process_dataset_ttm(base_dir)

Clean up dataset to prepare for augmentation

In [ ]:
# Iterate through all subdirectories in POP909
for root, dirs, files in os.walk(base_dir):
    for file in files:
        # Check if the file is a melody file
        if file.endswith("_melody.txt"):
            # Get the song ID from the subdirectory name
            song_id = os.path.basename(root)

            # Create the new filename with 6-digit song ID
            new_filename = f"{int(song_id):06d}_melody.txt"

            # Get the full paths
            old_path = os.path.join(root, file)
            new_path = os.path.join(base_dir, new_filename)

            # Move and rename the file
            shutil.move(old_path, new_path)

# Remove all subdirectories
for item in os.listdir(base_dir):
    item_path = os.path.join(base_dir, item)
    if os.path.isdir(item_path):
        shutil.rmtree(item_path)

print("Dataset cleaning completed.")

transposition, across an octave

In [ ]:
def transpose_melody(melody, shift):
  transposed = []
  for note in melody.split():
    if note.isdigit():
      transposed.append(str(int(note) + shift))
    else:
      transposed.append(note)
  return ' '.join(transposed)

# Get all melody files
melody_files = sorted([f for f in os.listdir(base_dir) if f.endswith('_melody.txt')])

# Start numbering from 910
new_file_number = 910

# Process each melody file
for file_name in melody_files:
  with open(os.path.join(base_dir, file_name), 'r') as file:
    original_melody = file.read().strip()

  # Generate transposed melodies
  for shift in range(-7, 5):
    if shift != 0:  # Skip the original melody (no transposition)
      transposed_melody = transpose_melody(original_melody, shift)

      # Create a new file name
      new_file_name = f'{new_file_number:06d}_melody.txt'
      new_file_path = os.path.join(base_dir, new_file_name)

      # Save the transposed melody
      with open(new_file_path, 'w') as new_file:
        new_file.write(transposed_melody)
        print(f"Saved song {new_file_name}")

      new_file_number += 1

print(f"Augmentation complete. {new_file_number - 910} new melody files created.")


zip up the dataset

In [ ]:
import shutil

def zip_folder(folder_name):
    # The output zip file will have the same name as the folder
    shutil.make_archive(folder_name, 'zip', folder_name)

# Usage
zip_folder("POP909")